# Линейная регрессия

Линейная регрессия является одним из самых простых обучающихся алгоритмов
в нашем инструментарии. Если вы когда-либо проходили вводный курс статистики
в колледже, вероятно, последней темой, которую вы рассматривали, была линейная
регрессия. На самом деле, она настолько проста, что иногда и вовсе не считается за
машинное самообучение! Что бы вы ни думали, факт остается фактом: линейная
регрессия — и ее расширения — продолжает оставаться распространенным и полезным методом предсказания, когда вектор целей является количественным
значением (например, цена дома, возраст). В этой главе мы рассмотрим различные
методы линейной регрессии (и некоторые расширения) для создания хорошо работающих предсказательных моделей.


Регрессия — способ выбрать из семейства функций ту, которая минимизирует функцию потерь. Последняя характеризует насколько сильно пробная функция отклоняется от значений в заданных точках. Если точки получены в эксперименте, они неизбежно содержат ошибку измерений, шум, поэтому разумнее требовать, чтобы функция передавала общую тенденцию, а не точно проходила через все точки. В каком-то смысле регрессия — это «интерполирующая аппроксимация»: мы хотим провести кривую как можно ближе к точкам и при этом сохранить ее максимально простой чтобы уловить общую тенденцию. За баланс между этими противоречивыми желаниями как-раз отвечает функция потерь (в английской литературе «loss function» или «cost function»).

**Подгонка прямой**

Требуется натренировать модель, представляющую линейную связь между признаком и вектором целей.

In [6]:
# Загрузить библиотеки
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
# Загрузить данные только с двумя признаками
california = fetch_california_housing()
features = california.data[:,0:2]
target = california.target
# Создать объект линейной регрессии
regression = LinearRegression()

# Выполнить подгонку линейной регрессии
model = regression.fit(features, target)

In [7]:
# Взглянуть на точку пересечения
model.intercept_

-0.10189032759082606

In [8]:
# Взглянуть на коэффициенты признаков
model.coef_

array([0.43169191, 0.01744134])

In [9]:
# Первое значение в векторе целей, умноженное на 1000
target[0]*1000

4526.0

In [10]:
# Предсказать целевое значение первого наблюдения, умноженное на 1000
model.predict(features)[0]*1000

4207.126263821179

Основным преимуществом линейной регрессии является ее интерпретируемость
в значительной степени потому, что модельные коэффициенты представляют собой эффект единичного изменения на вектор целей. Например, первый признак нашего
решения— это количество преступлений на одного жителя. Коэффициент этого
признака нашей модели составил -0.35. Это значит, что если мы умножим этот
коэффициент на 1000 (т. к. вектором целей является цена дома в тысячах долларов),
то у нас будет изменение в цене дома для каждого дополнительного преступления
на душу населения:


In [11]:
# Первый коэффициент, умноженный на 1000
model.coef_[0]*1000

431.6919075449536

Это говорит о том, что каждое преступление на душу населения снизит цену дома
примерно на $350!

**. Обработка интерактивных эффектов**

Задача

Имеется признак, влияние которого на целевую переменную зависит от другого
признака.

Решение

Создать член взаимодействия для захвата этой зависимости с помощью объекта
класса PolynomialFeatures библиотеки scikit-leam:

In [3]:
# Загрузить библиотеки
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import PolynomialFeatures
# Загрузить данные только с двумя признаками
california = fetch_california_housing()
features = california.data[:,0:2]
target = california.target
# Создать член, характеризующий взаимодействие между признаками
interaction = PolynomialFeatures(
degree=3, include_bias=False, interaction_only=True)
features_interaction = interaction.fit_transform(features)
# Создать объект линейной регрессии
regression = LinearRegression()
# Выполнить подгонку линейной регрессии
model = regression.fit(features_interaction, target)

Функция PolynomialFeatures сгенерировала новую матрицу показателей, состоящую из всех их полиномиальных комбинаций со степенью меньше или равной указанной (в нашем примере 2). Затем мы нормализовали эти данные и скормили их нашей модели.

Иногда влияние признака на целевую переменную по крайней мере частично зависит от другого признака. Например, представьте себе простой пример с кофе, где
у нас два бинарных признака — наличие сахара (sugar) и было ли выполнено перемешивание (stirred) — и требуется предсказать, сладкий ли кофе на вкус. Просто
положив сахар в кофе (sugar=l, stirred=0), мы не сделаем вкус кофе сладким (весь
сахар на дне!), а одно перемешивание кофе без добавления сахара (sugar=o,
stirred=i) не сделает его сладким. Как раз наоборот, именно взаимодействие брошенного в кофе сахара и перемешивания кофе (sugar=i, stirred=i) сделает вкус кофе сладким. Влияния сахара и перемешивания на сладость зависят друг от друга.
В этом случае мы говорим, что существует эффект взаимодействия между признаками sugar И stirred.


Мы можем учесть эффекты взаимодействия, включив новый признак, состоящий из
произведения соответствующих значений из взаимодействующих признаков:

У* = Ро + Р1*х + р2х, + рзх1х2 + е

где х1 и х2 — значения соответственно sugar и stirred; х1х2 — взаимодействие
между ними.

In [4]:
# Взглянуть на признаки для первого наблюдения
features[0]

array([ 8.3252, 41.    ])

Для того чтобы создать член, характеризующий взаимодействие между признаками, мы просто умножаем эти два значения между собой для каждого наблюдения:

In [7]:
# Импортировать библиотеку
import numpy as np
# Для каждого наблюдения перемножить значения
# первого и второго признаков
interaction_term = np.multiply(features[:, 0], features[:, 1])
#Затем мы можем взглянуть на член взаимодействия для первого наблюдения:
# Взглянуть на член взаимодействия для первого наблюдения
interaction_term[0]

341.33320000000003

Для создания членов взаимодействия с использованием объекта PolynomiaiFeatures
необходимо задать три важных параметра. Самый главный interaction_oniy=True
сообщает объекту PolynomiaiFeatures возвращать только члены взаимодействия
(а не полиномиальные признаки, которые мы обсудим в рецепте 13.3). По умолчанию класс PolynomiaiFeatures добавляет признак, содержащий те, которые называются смещением. Мы можем предотвратить это с помощью inciude_bias=Faise. Наконец, параметр degree определяет максимальное количество признаков для создания членов взаимодействия (в случае, если мы хотим создать член взаимодействия,
который является сочетанием трех признаков). Результат работы объекта класса
PolynomiaiFeatures можно увидеть из нашего решения, проверив, соответствуют ли
значения признаков первого наблюдения значению члена взаимодействия нашей
вручную рассчитанной версии:

In [8]:
# Взглянуть на значения для первого наблюдения
features_interaction[0]

array([  8.3252,  41.    , 341.3332])

** Подгонка нелинейной связи**

Задача

Требуется смоделировать нелинейную связь.

Решение

Создать полиномиальную регрессию путем включения полиномиальных признаков
в линейную регрессионную модель:

In [11]:
# Загрузить библиотеки
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import PolynomialFeatures
# Загрузить данные с одним признаком
california = fetch_california_housing()
features = california.data[:,0:1]
target = california.target
# Создать полиномиальные признаки хл2 и хл3
polynomial = PolynomialFeatures(degree=3, include_bias=False)
features_polynomial = polynomial.fit_transform(features)
# Создать объект линейной регрессии
regression = LinearRegression()
# Выполнить подгонку линейной регрессии
model = regression.fit(features_polynomial, target)

До сих пор мы обсуждали моделирование только линейных связей. Примером линейной связи может служить количество этажей здания и его высота. В линейной
регрессии мы исходим из того, что эффект количества этажей и высоты здания
примерно постоянен, а это означает, что 20-этажное здание будет примерно в два
раза выше, чем 10-этажное, которое будет примерно в два раза выше, чем 5-этажное здание. Однако многие представляющие интерес связи не являются строго
линейными.

Нередко требуется смоделировать нелинейную связь — например, связь между количеством часов, которые учащийся тратит на учебу, и оценкой, которую он получает за контрольную работу. Интуитивно мы можем себе представить, что в оценках между учащимися, которые учились в течение одного часа, и учащимися, которые вообще не учились, есть большая разница. Вместе с тем существует гораздо
меньшая разница в оценках между учащимся, который учился в течение 99 часов,
и учащимся, который учился в течение 100 часов. Влияние одного часа учебы на
итоговую оценку учащегося уменьшается по мере увеличения количества часов.
Полиномиальная регрессия — это расширение линейной регрессии, позволяющее
моделировать нелинейные связи. Для того чтобы создать полиномиальную регрессию, следует конвертировать линейную функцию, которую мы использовали в рецепте 13.1:

где d — степень полинома. Как использовать линейную регрессию для нелинейного признака? Ответ заключается в том, что мы ничего не изменяем в том, как
линейная регрессия подбирает модель, а только добавляем полиномиальные признаки. То есть, линейная регрессия не "знает", что х 2 является квадратичным преобразованием х. Она просто рассматривает его, как еще одну переменную.

Более практичное описание может лежать в порядке. Для того чтобы смоделировать нелинейные связи, мы можем создать новые признаки, которые возводят
существующий признак, х, в некоторую степень: х2, х3 и т. д. Чем больше этих
новых признаков мы добавим, тем более гибкой будет "линия", созданная нашей
моделью. Для того чтобы более четко отразить суть, представьте, что мы хотим
создать полином третьей степени. Для простоты мы сосредоточимся только на
одном наблюдении (первом наблюдении в наборе данных) х0:

In [12]:
# Взглянуть на первое наблюдение
features[0]

array([8.3252])

Для того чтобы создать полиномиальный признак, мы возведем первое значение
наблюдения во вторую степень — х^2:

In [13]:
# Взглянуть на первое наблюдение, возведенное во вторую степень, хА2
features[0]**2

array([69.30895504])

Включив все три признака (х, х2 и х3) в матрицу признаков, а затем выполнив
линейную регрессию, мы провели полиномиальную регрессию:

In [14]:
# Взглянуть на значения первого наблюдения для х, хА2 и хА3
features_polynomial[0]

array([  8.3252    ,  69.30895504, 577.0109125 ])

Полиномиальные признаки имеют два важных параметра. Во-первых, degree определяет максимальное число степеней для полиномиальных признаков. Например, degree=3 будет генерировать х2 и х3. Наконец, по умолчанию объект
Polynomial Features включает в себя признаки, содержащие одни единицы (называемые Смещением). Мы МОЖеМ удаЛИТЬ ИХ, установив include_bias=False.

**Снижение дисперсии
с помощью регуляризации**

Требуется уменьшить дисперсию линейной регрессионной модели.

Использовать обучающийся алгоритм, который включает сжимающий штраф (так
называемую регуляризацию), такой как гребневая регрессия и лассо-регрессия:

In [18]:
# Загрузить библиотеки
from sklearn.linear_model import Ridge
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# Загрузить данные
california = fetch_california_housing()
features = california.data
target = california.target
# Стандартизировать признаки
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)
# Создать объект гребневой регрессии со значением альфа
regression = Ridge(alpha=0.5)
# Выполнить подгонку линейной регрессии
model = regression.fit(features_standardized, target)

В стандартной линейной регрессии модель тренируется минимизировать сумму
квадратической ошибки (погрешности) между истинными у, и предсказываемыми у, целевыми значениями, или остаточную сумму квадратов (residual sum of
squares, RSS):
    
    RSS = sum(yi-<u>yi</u>)^2
Регуляризованные регрессионные ученики похожи, за исключением того, что они
пытаются минимизировать RSS и некий штраф за общий размер значений коэффициентов, называемый сжимающим штрафом, потому что он пытается "сжать" модель. Существуют два распространенных типа регуляризованных учеников для линейной регрессии: гребневая регрессия и лассо. Единственным формальным различием между ними является тип применяемого сжимающего штрафа. В гребневой
регрессии сжимающий штраф — это настроечный гиперпараметр, умноженный на
квадрат суммы всех коэффициентов:

    RSS + alpha* sum(beta** 2)
    
где betai — коэффициент j -го из р признаков; аlpha — гиперпараметр (обсуждается далее). Лассо-регрессия очень похожа на гребневую регрессию, за исключением того,
что сжимающий штраф — это настроечный параметр, умножаемый на сумму абсолютных значений всех коэффициентов:

    1/2n * RSS + alpha * sum(|<u>b</u>i|)



где п — количество наблюдений. Так какой же из них следует использовать? В качестве очень общего эмпирического правила нужно учитывать, что гребневая perрессия часто дает несколько лучшие предсказания, чем лассо, но лассо (по причинам, которые мы обсудим в рецепте 13.5) производит более интерпретируемые модели. Если мы хотим сбалансировать штрафные функции между гребнем и лассо,
мы можем использовать эластичную сеть, представляющую собой регрессионную
модель, в которую включены оба штрафа. Независимо от того, какой из них мы используем, гребневая регрессия и лассо-регрессия могут штрафовать большие или
сложные модели, включая значения коэффициентов в функцию потери, которую
мы пытаемся минимизировать.

Гиперпараметр а позволяет нам контролировать то, насколько мы штрафуем коэффициенты, где более высокие значения а создают более простые модели. Идеальное значение а должно быть настроено, как и любой другой гиперпараметр.
В библиотеке scikit-leam а устанавливается с помощью параметра alpha.

Библиотека scikit-leam включает класс Ridgecv, реализующий метод, который позволяет отбирать идеальное значение для а :

In [20]:
# Загрузить библиотеку
from sklearn.linear_model import RidgeCV
# Создать объект гребневой регрессии с тремя значениями alpha
regr_cv = RidgeCV(alphas=[0.1, 1.0, 10.0])
# Выполнить подгонку линейной регрессии
model_cv = regr_cv.fit(features_standardized, target)
# Взглянуть на коэффициенты
model_cv.coef_

array([ 0.8293461 ,  0.11939823, -0.26422311,  0.30398067, -0.00427544,
       -0.03936068, -0.8937389 , -0.86433656])

In [21]:
# Взглянуть на alpha
model_cv.alpha_

10.0

И последнее замечание: поскольку в линейной регрессии значение коэффициентов
частично определяется шкалой признака, а в регуляризованных моделях все коэффициенты суммируются вместе, перед тренировкой мы должны убедиться, что
стандартизировали признак.

 **Уменьшение количества признаков
с помощью лассо-регрессии**

Использовать лассо-регрессию:

In [23]:
# Загрузить библиотеки
from sklearn.linear_model import Lasso
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
# Загрузить данные
california = fetch_california_housing()
features = california.data
target = california.target
# Стандартизировать признаки
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)
# Создать объект лассо-регрессии со значением alpha
regression = Lasso(alpha=0.5)
# Выполнить подгонку линейной регрессии
model = regression.fit(features_standardized, target)

Одна из интересных характеристик штрафа лассо-регрессии состоит в том, что он
может сжимать коэффициенты модели до нуля, эффективно уменьшая количество
признаков в модели. Например, в нашем решении мы установили alpha на уровне
0.5 и видим, что многие коэффициенты равны 0. Иными словами, их соответствующие признаки в модели не используются:

In [25]:
# Взглянуть на коэффициенты
model.coef_

array([ 0.29398939,  0.        ,  0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        ])

In [28]:
# Создать лассо-регрессию с высоким alpha
regression_a10 = Lasso(alpha=10)
model_a10 = regression_a10.fit(features_standardized, target)
model_a10.coef_

array([ 0.,  0.,  0., -0., -0., -0., -0., -0.])

Однако, если мы увеличим а до гораздо более высокого значения, мы увидим, что
буквально ни один из признаков не используется

Практическая выгода от этого эффекта в следующем: он означает, что мы можем
включить в нашу матрицу признаков 100 признаков, а затем путем настройки
гиперпараметра а лассо-регрессии создать модель, которая использует только (например) 10 наиболее важных признаков. Это позволяет нам уменьшить дисперсию,
улучшая интерпретируемость нашей модели (поскольку меньшее количество признаков легче объяснить).